In [20]:
import pandas as pd
import re
import numpy as np

df=pd.read_csv("../data/donnees_dpe_rhone.csv",delimiter=",")

data=pd.DataFrame(df)
pd.set_option('display.max_columns', None)  # Affiche toutes les colonnes
data.head(10)

,configuration_installation_chauffage_n2,configuration_installation_chauffage_n1,conso_chauffage_installation_chauffage_n1,type_generateur_n1_ecs_n1,numero_voie_ban,score_ban,conso_chauffage_generateur_n1_installation_n2,conso_auxiliaires_ep,conso_chauffage_installation_chauffage_n2,deperditions_murs,cout_eclairage,conso_auxiliaires_ef,statut_geocodage,ventilation_posterieure_2012,cout_chauffage,conso_ecs_ef_energie_n3,conso_5_usages_par_m2_ep,date_etablissement_dpe,conso_ecs_ef_energie_n1,conso_ecs_ef_energie_n2,emission_ges_chauffage,description_installation_chauffage_n2,description_installation_chauffage_n1,qualite_isolation_plancher_haut_comble_amenage,conso_5_usages_par_m2_ef,cout_ecs_energie_n2,conso_chauffage_ef_energie_n1,cout_ecs_energie_n3,conso_chauffage_ef_energie_n2,presence_brasseur_air,qualite_isolation_menuiseries,conso_chauffage_ef_energie_n3,cout_total_5_usages_energie_n2,cout_total_5_usages_energie_n3,date_reception_dpe,cout_total_5_usages_energie_n1,cout_ecs_energie_n1,qualite_isolation_plancher_bas,isolation_toiture,modele_dpe,qualite_isolation_enveloppe,conso_chauffage_generateur_n1_installation_n1,type_energie_n1,emission_ges_eclairage,type_energie_n2,type_energie_n3,code_postal_ban,conso_5_usages_ef_energie_n3,emission_ges_ecs,conso_5_usages_ef_energie_n2,conso_5_usages_ef,conso_5_usages_ef_energie_n1,code_insee_ban,deperditions_planchers_bas,conso_5_usages_ep,date_fin_validite_dpe,deperditions_enveloppe,complement_adresse_batiment,code_region_ban,volume_stockage_generateur_n1_ecs_n1,surface_chauffee_installation_chauffage_n2,surface_chauffee_installation_chauffage_n1,version_dpe,besoin_ecs,coordonnee_cartographique_x_ban,type_generateur_chauffage_principal,type_energie_principale_ecs,apport_solaire_saison_chauffe,adresse_ban,deperditions_renouvellement_air,_rand,surface_habitable_desservie_par_installation_ecs_n1,production_electricite_pv_kwhep_par_an,nombre_niveau_logement,surface_habitable_logement,cout_ecs,type_installation_ecs_n1,emission_ges_5_usages_energie_n1,emission_ges_5_usages_energie_n2,apport_interne_saison_froide,emission_ges_5_usages_par_m2,description_generateur_chauffage_n1_installation_n1,apport_interne_saison_chauffe,apport_solaire_saison_froide,emission_ges_5_usages_energie_n3,type_generateur_n1_installation_n1,nombre_logements_desservis_par_installation_ecs_n1,type_generateur_n1_installation_n2,cout_auxiliaires,type_emetteur_installation_chauffage_n2,type_emetteur_installation_chauffage_n1,besoin_chauffage,configuration_installation_ecs_n1,description_installation_ecs_n1,description_generateur_chauffage_n1_installation_n2,classe_inertie_batiment,deperditions_ponts_thermiques,type_generateur_chauffage_principal_ecs,emission_ges_refroidissement,hauteur_sous_plafond,conso_chauffage_ef,nom_commune_ban,_geopoint,date_visite_diagnostiqueur,type_batiment,periode_construction,conso_ecs_ep,protection_solaire_exterieure,conso_ecs_ef,emission_ges_5_usages,date_derniere_modification_dpe,etiquette_ges,identifiant_ban,deperditions_baies_vitrees,type_energie_generateur_n1_ecs_n1,ubat_w_par_m2_k,numero_etage_appartement,nom_commune_brut,conso_ef_installation_ecs_n1,etiquette_dpe,description_generateur_n1_ecs_n1,code_departement_ban,type_installation_chauffage_n1,type_installation_chauffage_n2,methode_application_dpe,adresse_brut,logement_traversant,cout_total_5_usages,categorie_enr,conso_refroidissement_ef,conso_eclairage_ef,deperditions_planchers_hauts,zone_climatique,conso_ef_generateur_n1_ecs_n1,emission_ges_ecs_energie_n1,emission_ges_ecs_energie_n2,emission_ges_ecs_energie_n3,cout_refroidissement,conso_chauffage_ep,usage_generateur_n1_installation_n2,conso_eclairage_ep,usage_generateur_n1_installation_n1,nom_rue_ban,qualite_isolation_murs,type_installation_solaire_n1,classe_altitude,conso_refroidissement_ep,type_energie_principale_chauffage,numero_dpe,_i,emission_ges_chauffage_energie_n3,besoin_refroidissement,emission_ges_chauffage_energie_n2,indicateur_confort_ete,emission_ges_chauffage_ene

In [21]:
print(data.shape)

(256704, 202)


In [22]:
# 1️ Vérification des colonnes disponibles
print("\nColonnes disponibles :")
print(df.columns.tolist())


Colonnes disponibles :
['configuration_installation_chauffage_n2', 'configuration_installation_chauffage_n1', 'conso_chauffage_installation_chauffage_n1', 'type_generateur_n1_ecs_n1', 'numero_voie_ban', 'score_ban', 'conso_chauffage_generateur_n1_installation_n2', 'conso_auxiliaires_ep', 'conso_chauffage_installation_chauffage_n2', 'deperditions_murs', 'cout_eclairage', 'conso_auxiliaires_ef', 'statut_geocodage', 'ventilation_posterieure_2012', 'cout_chauffage', 'conso_ecs_ef_energie_n3', 'conso_5_usages_par_m2_ep', 'date_etablissement_dpe', 'conso_ecs_ef_energie_n1', 'conso_ecs_ef_energie_n2', 'emission_ges_chauffage', 'description_installation_chauffage_n2', 'description_installation_chauffage_n1', 'qualite_isolation_plancher_haut_comble_amenage', 'conso_5_usages_par_m2_ef', 'cout_ecs_energie_n2', 'conso_chauffage_ef_energie_n1', 'cout_ecs_energie_n3', 'conso_chauffage_ef_energie_n2', 'presence_brasseur_air', 'qualite_isolation_menuiseries', 'conso_chauffage_ef_energie_n3', 'cout_to

In [23]:
empty_cols = df.columns[df.isna().all()]
len(empty_cols), list(empty_cols)[:20]  # on affiche les 20 premières si la liste est longue


(0, [])

In [13]:
# Pourcentage de valeurs manquantes par colonne
missing_ratio = df.isna().mean().sort_values(ascending=False)
print(missing_ratio.head(100))  # affiche les 20 colonnes les plus vides


appartement_non_visite                                  0.988095
type_ventilation                                        0.988095
nombre_niveau_immeuble                                  0.988095
surface_ventilee                                        0.988095
facteur_couverture_solaire_installation_chauffage_n1    0.988095
                                                          ...   
type_generateur_chauffage_principal                     0.059524
type_installation_chauffage_n1                          0.059524
conso_chauffage_generateur_n1_installation_n1           0.059524
usage_generateur_n1_installation_n1                     0.059524
conso_chauffage_installation_chauffage_n1               0.059524
Length: 100, dtype: float64


In [25]:

empty_cols = df.columns[df.isna().all()]
print(f"Colonnes totalement vides supprimées ({len(empty_cols)}): {list(empty_cols)}")

df = df.dropna(axis=1, how="all")
print(f"✅ Nouveau shape après suppression des colonnes vides : {df.shape}")

# --- 3️⃣ Copier les données sources ---
raw = df.copy()
std = raw.copy()

print(f"✅ Toutes les colonnes non vides conservées : {len(std.columns)} colonnes")

# --- 4️⃣ Typage automatique des colonnes numériques ---
# On identifie les colonnes contenant des valeurs numériques possibles
num_candidates = std.select_dtypes(include=["number"]).columns.tolist()

# On force aussi la conversion des colonnes textuelles qui contiennent des chiffres
for c in std.columns:
    if std[c].dtype == "object":
        try:
            std[c] = pd.to_numeric(std[c], errors="ignore")
        except:
            pass

# --- 5️⃣ Conversion des dates si elles ressemblent à des dates ---
for c in std.columns:
    if re.search(r"date", c, re.IGNORECASE):
        std[c] = pd.to_datetime(std[c], errors="ignore")

# --- 6️⃣ Conversion de 'annee_construction' en entier (nullable) ---
if "annee_construction" in std.columns:
    std["annee_construction"] = pd.to_numeric(std["annee_construction"], errors="coerce").round().astype("Int64")

# --- 7️⃣ Inférence de l’année à partir de la période si manquante ---
def parse_periode_to_year(val):
    if pd.isna(val): 
        return np.nan
    s = str(val).lower().strip()
    m = re.match(r"(\d{4})\s*-\s*(\d{4})", s)
    if m:
        a1, a2 = int(m.group(1)), int(m.group(2))
        return int((a1 + a2) / 2)
    m = re.search(r"(\d{4})", s)
    if "avant" in s and m: return int(m.group(1))
    if "après" in s and m: return int(m.group(1))
    if m: return int(m.group(1))
    return np.nan

if "annee_construction" in std.columns and "periode_construction" in std.columns:
    mask_nan_year = std["annee_construction"].isna()
    std.loc[mask_nan_year, "annee_construction"] = std.loc[mask_nan_year, "periode_construction"].apply(parse_periode_to_year)
    std["annee_construction"] = std["annee_construction"].round().astype("Int64")

# --- 8️⃣ Nettoyage des valeurs textuelles ("nan", espaces, etc.) ---
std = std.replace(to_replace=[r"^\s*nan\s*$", r"^\s*NaN\s*$", r"^\s*None\s*$", r"^\s*$"], value=np.nan, regex=True)
for c in std.select_dtypes(include=["object"]).columns:
    std[c] = std[c].astype(str).str.strip()

# --- 9️⃣ Résumé du nettoyage ---
print("\n✅ Nettoyage terminé")
print(f"Nombre de colonnes conservées : {std.shape[1]}")
print(f"Nombre de lignes : {std.shape[0]}")
print("\nAperçu des valeurs manquantes :")
print(std.isna().sum().sort_values(ascending=False).head(10))

# --- 👀 Aperçu ---
std.head(10)


Colonnes totalement vides supprimées (0): []
✅ Nouveau shape après suppression des colonnes vides : (256704, 202)
✅ Toutes les colonnes non vides conservées : 202 colonnes


C:\Users\bah\AppData\Local\Temp\ipykernel_20172\3178484517.py:21: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  std[c] = pd.to_numeric(std[c], errors="ignore")
C:\Users\bah\AppData\Local\Temp\ipykernel_20172\3178484517.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  std[c] = pd.to_datetime(std[c], errors="ignore")
C:\Users\bah\AppData\Local\Temp\ipykernel_20172\3178484517.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  std[c] = pd.to_datetime(std[c], errors="ignore")
C:\Users\bah\AppData\Local\Temp\ipykernel_20172\3178484517.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime wi


✅ Nettoyage terminé
Nombre de colonnes conservées : 202
Nombre de lignes : 256704

Aperçu des valeurs manquantes :
appartement_non_visite                                  253648
surface_ventilee                                        253648
facteur_couverture_solaire_n1                           253648
facteur_couverture_solaire_installation_chauffage_n1    253648
nombre_niveau_immeuble                                  253648
production_ecs_solaire_installation_n1                  253648
cop_generateur_n1_ecs_n1                                250592
surface_habitable_immeuble                              250592
electricite_pv_autoconsommee                            247536
conso_refroidissement_annuel                            247536
dtype: int64


,configuration_installation_chauffage_n2,configuration_installation_chauffage_n1,conso_chauffage_installation_chauffage_n1,type_generateur_n1_ecs_n1,numero_voie_ban,score_ban,conso_chauffage_generateur_n1_installation_n2,conso_auxiliaires_ep,conso_chauffage_installation_chauffage_n2,deperditions_murs,cout_eclairage,conso_auxiliaires_ef,statut_geocodage,ventilation_posterieure_2012,cout_chauffage,conso_ecs_ef_energie_n3,conso_5_usages_par_m2_ep,date_etablissement_dpe,conso_ecs_ef_energie_n1,conso_ecs_ef_energie_n2,emission_ges_chauffage,description_installation_chauffage_n2,description_installation_chauffage_n1,qualite_isolation_plancher_haut_comble_amenage,conso_5_usages_par_m2_ef,cout_ecs_energie_n2,conso_chauffage_ef_energie_n1,cout_ecs_energie_n3,conso_chauffage_ef_energie_n2,presence_brasseur_air,qualite_isolation_menuiseries,conso_chauffage_ef_energie_n3,cout_total_5_usages_energie_n2,cout_total_5_usages_energie_n3,date_reception_dpe,cout_total_5_usages_energie_n1,cout_ecs_energie_n1,qualite_isolation_plancher_bas,isolation_toiture,modele_dpe,qualite_isolation_enveloppe,conso_chauffage_generateur_n1_installation_n1,type_energie_n1,emission_ges_eclairage,type_energie_n2,type_energie_n3,code_postal_ban,conso_5_usages_ef_energie_n3,emission_ges_ecs,conso_5_usages_ef_energie_n2,conso_5_usages_ef,conso_5_usages_ef_energie_n1,code_insee_ban,deperditions_planchers_bas,conso_5_usages_ep,date_fin_validite_dpe,deperditions_enveloppe,complement_adresse_batiment,code_region_ban,volume_stockage_generateur_n1_ecs_n1,surface_chauffee_installation_chauffage_n2,surface_chauffee_installation_chauffage_n1,version_dpe,besoin_ecs,coordonnee_cartographique_x_ban,type_generateur_chauffage_principal,type_energie_principale_ecs,apport_solaire_saison_chauffe,adresse_ban,deperditions_renouvellement_air,_rand,surface_habitable_desservie_par_installation_ecs_n1,production_electricite_pv_kwhep_par_an,nombre_niveau_logement,surface_habitable_logement,cout_ecs,type_installation_ecs_n1,emission_ges_5_usages_energie_n1,emission_ges_5_usages_energie_n2,apport_interne_saison_froide,emission_ges_5_usages_par_m2,description_generateur_chauffage_n1_installation_n1,apport_interne_saison_chauffe,apport_solaire_saison_froide,emission_ges_5_usages_energie_n3,type_generateur_n1_installation_n1,nombre_logements_desservis_par_installation_ecs_n1,type_generateur_n1_installation_n2,cout_auxiliaires,type_emetteur_installation_chauffage_n2,type_emetteur_installation_chauffage_n1,besoin_chauffage,configuration_installation_ecs_n1,description_installation_ecs_n1,description_generateur_chauffage_n1_installation_n2,classe_inertie_batiment,deperditions_ponts_thermiques,type_generateur_chauffage_principal_ecs,emission_ges_refroidissement,hauteur_sous_plafond,conso_chauffage_ef,nom_commune_ban,_geopoint,date_visite_diagnostiqueur,type_batiment,periode_construction,conso_ecs_ep,protection_solaire_exterieure,conso_ecs_ef,emission_ges_5_usages,date_derniere_modification_dpe,etiquette_ges,identifiant_ban,deperditions_baies_vitrees,type_energie_generateur_n1_ecs_n1,ubat_w_par_m2_k,numero_etage_appartement,nom_commune_brut,conso_ef_installation_ecs_n1,etiquette_dpe,description_generateur_n1_ecs_n1,code_departement_ban,type_installation_chauffage_n1,type_installation_chauffage_n2,methode_application_dpe,adresse_brut,logement_traversant,cout_total_5_usages,categorie_enr,conso_refroidissement_ef,conso_eclairage_ef,deperditions_planchers_hauts,zone_climatique,conso_ef_generateur_n1_ecs_n1,emission_ges_ecs_energie_n1,emission_ges_ecs_energie_n2,emission_ges_ecs_energie_n3,cout_refroidissement,conso_chauffage_ep,usage_generateur_n1_installation_n2,conso_eclairage_ep,usage_generateur_n1_installation_n1,nom_rue_ban,qualite_isolation_murs,type_installation_solaire_n1,classe_altitude,conso_refroidissement_ep,type_energie_principale_chauffage,numero_dpe,_i,emission_ges_chauffage_energie_n3,besoin_refroidissement,emission_ges_chauffage_energie_n2,indicateur_confort_ete,emission_ges_chauffage_ene

### Convertir les "nan" textuels en valeurs manquantes réelles

In [26]:
# remplace les "nan", "NaN", "None", "" par de vrais NaN
std = std.replace(to_replace=[r"^\s*nan\s*$", r"^\s*NaN\s*$", r"^\s*None\s*$", r"^\s*$"], value=np.nan, regex=True)


In [28]:
std.head(3)

,configuration_installation_chauffage_n2,configuration_installation_chauffage_n1,conso_chauffage_installation_chauffage_n1,type_generateur_n1_ecs_n1,numero_voie_ban,score_ban,conso_chauffage_generateur_n1_installation_n2,conso_auxiliaires_ep,conso_chauffage_installation_chauffage_n2,deperditions_murs,cout_eclairage,conso_auxiliaires_ef,statut_geocodage,ventilation_posterieure_2012,cout_chauffage,conso_ecs_ef_energie_n3,conso_5_usages_par_m2_ep,date_etablissement_dpe,conso_ecs_ef_energie_n1,conso_ecs_ef_energie_n2,emission_ges_chauffage,description_installation_chauffage_n2,description_installation_chauffage_n1,qualite_isolation_plancher_haut_comble_amenage,conso_5_usages_par_m2_ef,cout_ecs_energie_n2,conso_chauffage_ef_energie_n1,cout_ecs_energie_n3,conso_chauffage_ef_energie_n2,presence_brasseur_air,qualite_isolation_menuiseries,conso_chauffage_ef_energie_n3,cout_total_5_usages_energie_n2,cout_total_5_usages_energie_n3,date_reception_dpe,cout_total_5_usages_energie_n1,cout_ecs_energie_n1,qualite_isolation_plancher_bas,isolation_toiture,modele_dpe,qualite_isolation_enveloppe,conso_chauffage_generateur_n1_installation_n1,type_energie_n1,emission_ges_eclairage,type_energie_n2,type_energie_n3,code_postal_ban,conso_5_usages_ef_energie_n3,emission_ges_ecs,conso_5_usages_ef_energie_n2,conso_5_usages_ef,conso_5_usages_ef_energie_n1,code_insee_ban,deperditions_planchers_bas,conso_5_usages_ep,date_fin_validite_dpe,deperditions_enveloppe,complement_adresse_batiment,code_region_ban,volume_stockage_generateur_n1_ecs_n1,surface_chauffee_installation_chauffage_n2,surface_chauffee_installation_chauffage_n1,version_dpe,besoin_ecs,coordonnee_cartographique_x_ban,type_generateur_chauffage_principal,type_energie_principale_ecs,apport_solaire_saison_chauffe,adresse_ban,deperditions_renouvellement_air,_rand,surface_habitable_desservie_par_installation_ecs_n1,production_electricite_pv_kwhep_par_an,nombre_niveau_logement,surface_habitable_logement,cout_ecs,type_installation_ecs_n1,emission_ges_5_usages_energie_n1,emission_ges_5_usages_energie_n2,apport_interne_saison_froide,emission_ges_5_usages_par_m2,description_generateur_chauffage_n1_installation_n1,apport_interne_saison_chauffe,apport_solaire_saison_froide,emission_ges_5_usages_energie_n3,type_generateur_n1_installation_n1,nombre_logements_desservis_par_installation_ecs_n1,type_generateur_n1_installation_n2,cout_auxiliaires,type_emetteur_installation_chauffage_n2,type_emetteur_installation_chauffage_n1,besoin_chauffage,configuration_installation_ecs_n1,description_installation_ecs_n1,description_generateur_chauffage_n1_installation_n2,classe_inertie_batiment,deperditions_ponts_thermiques,type_generateur_chauffage_principal_ecs,emission_ges_refroidissement,hauteur_sous_plafond,conso_chauffage_ef,nom_commune_ban,_geopoint,date_visite_diagnostiqueur,type_batiment,periode_construction,conso_ecs_ep,protection_solaire_exterieure,conso_ecs_ef,emission_ges_5_usages,date_derniere_modification_dpe,etiquette_ges,identifiant_ban,deperditions_baies_vitrees,type_energie_generateur_n1_ecs_n1,ubat_w_par_m2_k,numero_etage_appartement,nom_commune_brut,conso_ef_installation_ecs_n1,etiquette_dpe,description_generateur_n1_ecs_n1,code_departement_ban,type_installation_chauffage_n1,type_installation_chauffage_n2,methode_application_dpe,adresse_brut,logement_traversant,cout_total_5_usages,categorie_enr,conso_refroidissement_ef,conso_eclairage_ef,deperditions_planchers_hauts,zone_climatique,conso_ef_generateur_n1_ecs_n1,emission_ges_ecs_energie_n1,emission_ges_ecs_energie_n2,emission_ges_ecs_energie_n3,cout_refroidissement,conso_chauffage_ep,usage_generateur_n1_installation_n2,conso_eclairage_ep,usage_generateur_n1_installation_n1,nom_rue_ban,qualite_isolation_murs,type_installation_solaire_n1,classe_altitude,conso_refroidissement_ep,type_energie_principale_chauffage,numero_dpe,_i,emission_ges_chauffage_energie_n3,besoin_refroidissement,emission_ges_chauffage_energie_n2,indicateur_confort_ete,emission_ges_chauffage_ene

In [31]:
# --- 🔟 Sauvegarde du jeu de données nettoyé ---
std.to_csv("../data/donnees_dpe_rhone_clean.csv", index=False)
print("\n💾 Données nettoyées enregistrées sous : donnees_dpe_rhone_clean.csv")


💾 Données nettoyées enregistrées sous : donnees_dpe_rhone_clean.csv
